In [151]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [152]:
def load_data(data_path):
    with open(data_path, 'r') as fp:
        data = json.load(fp)
    X = np.array(data['mfcc'])
    y = np.array(data['labels'])
    return X, y

In [153]:
X, y = load_data('json data.json')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [154]:
X_train.shape

(100, 92, 12)

In [155]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [156]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': [2, 3, 5, 7, 8, 9, 11, 19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

clf = GridSearchCV(KNeighborsClassifier(), grid_params, cv = 3, scoring = 'accuracy')
clf.fit(X_train, y_train)
print(f'best params: {clf.best_params_}')
print(f'accuracy: {clf.best_score_}')
print(f'classifier: {clf.best_estimator_}')

best params: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'}
accuracy: 0.9699940582293524
classifier: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance')


In [157]:
y_pred = clf.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         6
           3       0.17      0.50      0.25         2
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00        10
           6       1.00      0.55      0.71        11
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         9

    accuracy                           0.91        67
   macro avg       0.90      0.90      0.89        67
weighted avg       0.96      0.91      0.92        67



In [149]:
y_test

array([9, 3, 7, 3, 5, 2, 0, 0, 4, 8, 2, 5, 9, 5, 4, 2, 4, 5, 8, 5, 6, 6,
       9, 9, 2, 9, 1, 8, 9, 1, 0, 6, 7, 1, 3, 2, 1, 3, 5, 0, 7, 9, 8, 0,
       3, 8, 2, 9, 1, 4, 5, 7, 5, 6, 1, 5, 7, 5, 6, 6, 0, 7, 9, 1, 4, 3,
       0])

In [158]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred, y_test))

[[ 6  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0]
 [ 0  0  6  0  0  0  0  0  0  0]
 [ 1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0  5  0  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  0  0  0  9]]


In [160]:
import librosa
category = [
        "cua",
        "dong_co_may_bay_cat_canh",
        "dong_co_xe_may",
        "khoan",
        "may_bom_hoi",
        "may_cay",
        "may_dem_tien",
        "may_hut_bui",
        "may_say",
        "quat_suoi"
]
file = 'raw data/may_cay/may_cay_13(mp3cut.net).wav'
signal, sr = librosa.load(file, sr = 22050)
mfcc = librosa.feature.mfcc(signal, sr = 22050, n_mfcc = 12, n_fft = 2048, hop_length = 512)
mfcc = mfcc.T
mfcc_pad = np.zeros((92, 12))
mfcc_pad[:mfcc.shape[0], :] = mfcc
mfcc_pad = mfcc_pad.reshape(1, -1)
print(category[clf.predict(mfcc_pad)[0]])

may_cay
